In [2]:
import pandas as pd
pd.set_option("display.max_rows", 5)
reviews = pd.read_csv("input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)

In [3]:
# median of the points column in the dataframe

In [6]:
median_points = reviews.points.median()

In [7]:
median_points

88.0

In [8]:
# unique countries

In [12]:
countries = reviews.country.unique()
countries

array(['Italy', 'Portugal', 'US', 'Spain', 'France', 'Germany',
       'Argentina', 'Chile', 'Australia', 'Austria', 'South Africa',
       'New Zealand', 'Israel', 'Hungary', 'Greece', 'Romania', 'Mexico',
       'Canada', nan, 'Turkey', 'Czech Republic', 'Slovenia',
       'Luxembourg', 'Croatia', 'Georgia', 'Uruguay', 'England',
       'Lebanon', 'Serbia', 'Brazil', 'Moldova', 'Morocco', 'Peru',
       'India', 'Bulgaria', 'Cyprus', 'Armenia', 'Switzerland',
       'Bosnia and Herzegovina', 'Ukraine', 'Slovakia', 'Macedonia',
       'China', 'Egypt'], dtype=object)

# how often does each country appear in the dataset?

In [17]:
reviews.country.describe()

count     129908
unique        43
top           US
freq       54504
Name: country, dtype: object

In [73]:
# len(reviews[reviews.country == 'Italy'])

In [72]:
# reviews_per_country = pd.Series(map(lambda country: len(reviews[reviews.country == country]), reviews.country.unique()))

In [74]:
reviews.country.value_counts()

US          54504
France      22093
            ...  
China           1
Slovakia        1
Name: country, Length: 43, dtype: int64

## 4.
Create variable `centered_price` containing a version of the `price` column with the mean price subtracted.

(Note: this 'centering' transformation is a common preprocessing step before applying various machine learning algorithms.) 

In [76]:
centered_price = reviews.price - reviews.price.mean()
centered_price

0               NaN
1        -20.363389
            ...    
129969    -3.363389
129970   -14.363389
Name: price, Length: 129971, dtype: float64

## 5.
I'm an economical wine buyer. Which wine is the "best bargain"? Create a variable `bargain_wine` with the title of the wine with the highest points-to-price ratio in the dataset.

In [81]:
(reviews.points/reviews.price).max()

21.5

In [91]:
bargain_wine = reviews.loc[(reviews.points/reviews.price)==(reviews.points/reviews.price).max()].title

In [92]:
bargain_wine

64590                         Bandit NV Merlot (California)
126096    Cramele Recas 2011 UnWineD Pinot Grigio (Viile...
Name: title, dtype: object

In [94]:
bargain_wine = reviews.loc[(reviews.points/reviews.price).idxmax()].title

In [95]:
bargain_wine

'Bandit NV Merlot (California)'

## 6.
There are only so many words you can use when describing a bottle of wine. Is a wine more likely to be "tropical" or "fruity"? Create a Series `descriptor_counts` counting how many times each of these two words appears in the `description` column in the dataset.

In [96]:
reviews.head(1)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia


In [111]:
['tropical' in description for description in reviews.description].count(True)

3607

In [108]:
descriptor_counts = pd.Series({'tropical':['tropical' in description for description in reviews.description].count(True), 
                              'fruity':['fruity' in description for description in reviews.description].count(True)})

In [109]:
descriptor_counts

tropical    3607
fruity      9090
dtype: int64

In [ ]:
# Solution:

n_trop = reviews.description.map(lambda desc: "tropical" in desc).sum()
n_fruity = reviews.description.map(lambda desc: "fruity" in desc).sum()
descriptor_counts = pd.Series([n_trop, n_fruity], index=['tropical', 'fruity'])

## 7.
We'd like to host these wine reviews on our website, but a rating system ranging from 80 to 100 points is too hard to understand - we'd like to translate them into simple star ratings. A score of 95 or higher counts as 3 stars, a score of at least 85 but less than 95 is 2 stars. Any other score is 1 star.

Also, the Canadian Vintners Association bought a lot of ads on the site, so any wines from Canada should automatically get 3 stars, regardless of points.

Create a series `star_ratings` with the number of stars corresponding to each review in the dataset.

In [123]:
def trans(row):
    country = row.country
    point = row.points
    
    if country == 'Canada':
        return 3
    else:
        if point >= 95:
            return 3
        elif 85 <= point <95:
            return 2
        else:
            return 1

star_ratings = reviews.apply(trans, axis = 'columns' )
star_ratings

0         2
1         2
         ..
129969    2
129970    2
Length: 129971, dtype: int64